In [1]:
# Install all required libraries

import sys
!{sys.executable} -m pip install pulp 

import pulp
# Import PuLP modeler functions
from pulp import * 

import numpy as np
import math as math
import networkx as nx

## The Decision Variables and Objective Function 

In [2]:
problem = LpProblem("CMST_Problem",LpMinimize)

In [3]:
# Graph of edges. The edges are the decision variables
edges_weights = np.array([
 [0,7,9,14,0,15,5,1,0], #root
 [0,0,16,0,15,0,0,0,0], 
 [0,16,0,0,0,3,0,0,0],
 [0,0,0,0,0,0,2,11,6],
 [0,15,0,0,0,0,0,10,4],
 [0,3,0,0,0,0,8,0,0],
 [0,0,0,2,0,8,0,0,0],
 [0,0,0,11,10,0,0,0,0],
 [0,0,0,6,4,0,0,0,0]
])

edges = ['RA', 'RB', 'RC', 'RE', 'RF', 'RG', 'AB', 'AD', 'BA', 'BE', 'CF', 'CG', 'CH', 'DA', 'DF', 'DG', 'EA', 'EF', 'FC', 'FE', 'GC', 'GD', 'HC', 'HD']
numOfRows = np.size(edges_weights, 0)
numOfCols = np.size(edges_weights, 1)

# For every entry in edges_weights is a decision variable
temp = []
for i in range(0,numOfRows):
    for j in range(0,numOfCols):
        if edges_weights[i][j] != 0:
            temp.append(edges_weights[i][j])

edge_costs = {}
for i in range(0, len(edges)):
    edge_costs[edges[i]] = temp[i]

edge_vars = LpVariable.dicts("edge",edges,lowBound=0)

# Capacity constraint
K = 3

## OBJECTIVE FUNCTION
problem += lpSum([edge_vars[i]*edge_costs[i] for i in edge_costs])


In [4]:
# Given sets V, V+, d, and given adjacency matrix edges and edge weight matrix edges_weights and capacity constraint K


# For the set V and V+, we can make it a list of numbers from 0 to n-1 and 1 to n-1, with n being the number of 
# rows in the adjacency matrix
# example: V= [0,1,2,3,4,5,6, ..., n-1] and V+ = [1,2,3,4,5,6, ..., n-1].

V = []
V_terminal = []

for i in range(0, numOfRows):
    if i != 0:
        V_terminal.append(i)
    V.append(i)

# For the first constraint, we want the sum of all elements in each column of E, except column 0, to be equal to 1
# i.e. the sum of all elements in a column j must equal 1 for all j in V+


# For the second constraint, we let d be a list of integers of size |V+|, with the entries being the demands of each node
# d = [4,7,2,3,4,9,...]

# Here the nodes are unit weight.
d = []
for i in range(0, numOfRows-1):
    d.append(1)
print(d)

# To get the number of subtrees in the graph, we take the sum of all "gates"(edges connecting the root node to another
# node). Thus, we would take the sum of all elements in the root node row of the adjacency matrix.
# example: if the row for v0 = [0,1,1,0,0,0,1,0,...], we would sum the elements n_s = (0+1+1+0+0+0+1+0+...)

gates = []
for i in range(0,numOfCols):
    if edges_weights[0][i] != 0:
        gates.append(1)

# Now, we want to sum all elements in d and divide that sum by the capacity constraint K. Afterwards, we apply the
# the ceiling fn to this expression in order to get the minimum number of subtrees necessary to satisfy the capacity 
# constraint.


# the second constraint would be the number of subtrees >= the minimium number of subtrees necessary for K
# example: n_s >= min_s_K
problem += lpSum([gates[i] for i in range(0, len(gates))]) >= math.ceil(sum(d)/K)
print(problem)


[1, 1, 1, 1, 1, 1, 1, 1]
CMST_Problem:
MINIMIZE
16*edge_AB + 15*edge_AD + 16*edge_BA + 3*edge_BE + 2*edge_CF + 11*edge_CG + 6*edge_CH + 15*edge_DA + 10*edge_DF + 4*edge_DG + 3*edge_EA + 8*edge_EF + 2*edge_FC + 8*edge_FE + 11*edge_GC + 10*edge_GD + 6*edge_HC + 4*edge_HD + 7*edge_RA + 9*edge_RB + 14*edge_RC + 15*edge_RE + 5*edge_RF + 1*edge_RG + 0
SUBJECT TO
_C1:0 >= -3

VARIABLES
edge_AB Continuous
edge_AD Continuous
edge_BA Continuous
edge_BE Continuous
edge_CF Continuous
edge_CG Continuous
edge_CH Continuous
edge_DA Continuous
edge_DF Continuous
edge_DG Continuous
edge_EA Continuous
edge_EF Continuous
edge_FC Continuous
edge_FE Continuous
edge_GC Continuous
edge_GD Continuous
edge_HC Continuous
edge_HD Continuous
edge_RA Continuous
edge_RB Continuous
edge_RC Continuous
edge_RE Continuous
edge_RF Continuous
edge_RG Continuous

